In [31]:
import os
from dotenv import load_dotenv
# api_key = os.environ.get("OPENAI_API_KEY")
# os.environ['OPENAI_API_KEY'] = api_key
load_dotenv()

True

In [19]:
from langchain import OpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate

In [20]:
import os
import json
import sys


def file_reader(path: str, ) -> str:
    fname = os.path.join(path)
    with open(fname, 'r') as f:
        system_message = f.read()
    return system_message
            
def get_prompt():
    prompt_message = file_reader("../prompts/prompt_generation_template.txt")
    prompt = str(prompt_message)
    return prompt

## Prompt Generator Template

In [21]:
RAG_PROMPT_TEMPLATE = get_prompt()

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)
rag_prompt

ChatPromptTemplate(input_variables=['context', 'output'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'output'], template='Your task is to formulate exactly 5 questions from given context and provide the answer to each one.\n\nEnd each question with a \'?\' character and then in a newline write the answer to that question using only \nthe context provided.\nThe output MUST BE in a json format. \n\nexample:\n[\n{{\n    "user": "What is the name of the company?",\n    "assistant": "Google"\n}},\n{{\n    "user": "What is the name of the CEO?",\n    "assistant": "Sundar Pichai"\n}}\n]\n\nEach question must start with "user:".\nEach answer must start with "assistant:".\n\n\nThe question must satisfy the rules given below:\n1.The question should make sense to humans even when read without the given context.\n2.The question should be fully answered from the given context.\n3.The question should be framed from a part of context that contains important

## Embeddings and Vector Store

In [22]:
loader = TextLoader('./week_6_challenge_doc.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 200, chunk_overlap=50, model_name = "gpt-4-1106-preview")
texts  = text_splitter.split_documents(documents)

llm = OpenAI(temperature=0)

embeddings = OpenAIEmbeddings()
store = Chroma.from_documents(texts,embeddings, collection_name="challenge_document")
len(texts)

24

## Stringoutput Parser

In [23]:
from langchain.schema import StrOutputParser

str_output_parser = StrOutputParser()

## Setting up RAG chain

In [24]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = store.as_retriever()

entry_point_and_retriever = RunnableParallel(
    {
        "context" : retriever,
        "output" : RunnablePassthrough()
    }
)

rag_chain = entry_point_and_retriever | rag_prompt | llm | str_output_parser

In [25]:
rag_chain.invoke('i want to know the goals of the challenge')

"\n\nuser: What is the goal of the challenge?\nassistant: The goal of the challenge is to support and reward expertise in different parts of the Machine learning engineering toolbox.\n\nuser: What are the fundamental tasks for this week's challenge?\nassistant: The fundamental tasks for this week's challenge are understanding prompt engineering tools and concepts, familiarizing with language models, developing a plan for prompt generation and testing, and setting up a development environment.\n\nuser: What are the key performance indicators for this challenge?\nassistant: The key performance indicators for this challenge are understanding prompt ranking, understanding prompt matching, and ability to reuse previous knowledge.\n\nuser: What are the deliverables for this challenge?\nassistant: The deliverables for this challenge include a PDF document stored in Google Drive or a published blog link, a link to the code repository on GitHub, and an interim submission by Wednesday 8pm UTC.\n